### Grafana
##### Existing Folder List

In [ ]:
import requests
import botoplus.organization as _org

selected = _org.selected()

ssm = selected.client('ssm')
grafana_url = ssm.get_parameter(Name='/grafana/url', WithDecryption=False)['Parameter']['Value']
grafana_token = ssm.get_parameter(Name='/grafana/token', WithDecryption=True)['Parameter']['Value']

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": "Bearer "+grafana_token
}
url = grafana_url+'/api/folders?query=json'
response = requests.get(url, headers=headers)
j = response.json()

existing_folders = []

for folder in j:
    existing_folders.append(folder['title'])
    print(folder['title'])

##### Existing Account List

In [ ]:
import botoplus.organization as _org

accounts = _org.accounts()

create_folders = []

for account in accounts:
    folder = account['awsalias']+' - '+str(account['awsaccount'])
    if folder not in existing_folders:
        create_folders.append(folder)
        print(folder)

##### New Folder Creation

In [ ]:
import requests
import botoplus.organization as _org

selected = _org.selected()

ssm = selected.client('ssm')
grafana_url = ssm.get_parameter(Name='/grafana/url', WithDecryption=False)['Parameter']['Value']
grafana_token = ssm.get_parameter(Name='/grafana/token', WithDecryption=True)['Parameter']['Value']

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": "Bearer "+grafana_token
}
url = grafana_url+'/api/folders?query=json'

for folder in create_folders:
    if folder not in existing_folders:
        data = {
            "title": folder
        }
        r = requests.post(url, headers=headers, json=data)
        print(r.json())

##### Folder List

In [ ]:
import requests
import botoplus.organization as _org

selected = _org.selected()

ssm = selected.client('ssm')
grafana_url = ssm.get_parameter(Name='/grafana/url', WithDecryption=False)['Parameter']['Value']
grafana_token = ssm.get_parameter(Name='/grafana/token', WithDecryption=True)['Parameter']['Value']

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": "Bearer "+grafana_token
}
url = grafana_url+'/api/folders?query=json'
response = requests.get(url, headers=headers)
j = response.json()

folder_list = []

for folder in j:
    parsed = folder['title'].split(' - ')
    if len(parsed) == 2:
        temp = {}
        temp['uid'] = folder['uid']
        temp['awsalias'] = parsed[0]
        temp['awsaccount'] = str(parsed[1])
        folder_list.append(temp)
        print(temp)

##### Data Sources

In [ ]:
import requests
import botoplus.organization as _org

selected = _org.selected()

ssm = selected.client('ssm')
grafana_url = ssm.get_parameter(Name='/grafana/url', WithDecryption=False)['Parameter']['Value']
grafana_token = ssm.get_parameter(Name='/grafana/token', WithDecryption=True)['Parameter']['Value']

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": "Bearer "+grafana_token
}
url = grafana_url+'/api/datasources?query=json'
response = requests.get(url, headers=headers)
j = response.json()

data_sources = []

for source in j:
    temp = {}
    temp['uid'] = source['uid']
    temp['awsaccount'] = str(source['jsonData']['accountId'])
    temp['region'] = source['jsonData']['defaultRegion']
    data_sources.append(temp)
    print(temp)

##### Provisioning Format Export

In [ ]:
import json
import requests
import botoplus.organization as _org

selected = _org.selected()

ssm = selected.client('ssm')
grafana_url = ssm.get_parameter(Name='/grafana/url', WithDecryption=False)['Parameter']['Value']
grafana_token = ssm.get_parameter(Name='/grafana/token', WithDecryption=True)['Parameter']['Value']

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": "Bearer "+grafana_token
}
url = grafana_url+'/api/v1/provisioning/alert-rules/export?query=json'

response = requests.get(url, headers=headers)
j = response.json()

example_title = 'Test'
update_value = 'CHANGEIT'

for item in j['groups']:
    for rule in item['rules']:
        if rule['title'] == example_title:

            url = grafana_url+'/api/v1/provisioning/alert-rules/'+rule['uid']+'/export'
            response = requests.get(url, headers=headers)
            j = response.json()

            j = j['groups'][0]['rules'][0]
            j.pop('uid', None)
            j['orgID'] = 1
            j['data'][0]['model']['logGroups'] = []

            j['title'] = update_value
            j['folderUID'] = update_value
            j['ruleGroup'] = update_value
            j['data'][0]['datasourceUid'] = update_value
            j['data'][0]['model']['datasource']['uid'] = update_value
            j['data'][0]['model']['dimensions']['FunctionName'] = update_value
            j['data'][0]['model']['sql']['where']['expressions'][0]['operator']['value'] = update_value
            j['data'][0]['model']['sqlExpression'] = "SELECT SUM(Errors) FROM \"AWS/Lambda\" WHERE FunctionName = '"+update_value+"'"

            with open(example_title+'.json', 'w') as f:
                json.dump(j, f)

            print(json.dumps(j, indent=4))
            break

##### Lambda Functions

In [ ]:
import botoplus.organization as _org

display_denied = False

accounts = _org.accounts()

lambda_functions = []

for account in accounts:
    print(str(account['awsaccount'])+' '+account['awsalias'])
    sessions = _org.sessions(account['awsaccount'])
    ec2 = sessions.client('ec2')
    regions = ec2.describe_regions()

    for region in regions['Regions']:
        try:
            lambda_region = sessions.client('lambda', region_name = region['RegionName'])
            paginator = lambda_region.get_paginator('list_functions')
            response_iterator = paginator.paginate()
            for page in response_iterator:
                if len(page['Functions']) > 0:
                    for item in page['Functions']:
                        temp = {}
                        temp['account'] = str(account['awsaccount'])
                        temp['alias'] = account['awsalias']
                        temp['name'] = item['FunctionName']
                        temp['region'] = region['RegionName']
                        lambda_functions.append(temp)
        except:
            if display_denied == True:
                print(' - '+region['RegionName']+' DENIED')
            pass

print('Lambda Count: '+str(len(lambda_functions)))

##### Alert List

In [ ]:
import requests
import botoplus.organization as _org

selected = _org.selected()

ssm = selected.client('ssm')
grafana_url = ssm.get_parameter(Name='/grafana/url', WithDecryption=False)['Parameter']['Value']
grafana_token = ssm.get_parameter(Name='/grafana/token', WithDecryption=True)['Parameter']['Value']

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": "Bearer "+grafana_token
}
url = grafana_url+'/api/v1/provisioning/alert-rules?query=json'

response = requests.get(url, headers=headers)
j = response.json()

alert_rules = []

for alert in j:
    temp = {}
    temp['uid'] = alert['uid']
    temp['title'] = alert['title']
    temp['ruleGroup'] = alert['ruleGroup']
    temp['folderUID'] = alert['folderUID']
    alert_rules.append(temp)
    print(temp)

##### New Alert Creation

In [ ]:
import json
import requests
import botoplus.organization as _org

def alertuid(title, region, folderuid):
    for alert in alert_rules:
        if alert['title'] == title and alert['ruleGroup'] == region and alert['folderUID'] == folderuid:
            return alert['uid']
    return None

def folderuid(awsaccount):
    for folder in folder_list:
        if folder['awsaccount'] == awsaccount:
            return folder['uid']
    return None

def sourceuid(awsaccount, region):
    for source in data_sources:
        if source['awsaccount'] == awsaccount and source['region'] == region:
            return source['uid']
    return None

selected = _org.selected()

ssm = selected.client('ssm')
grafana_url = ssm.get_parameter(Name='/grafana/url', WithDecryption=False)['Parameter']['Value']
grafana_token = ssm.get_parameter(Name='/grafana/token', WithDecryption=True)['Parameter']['Value']

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": "Bearer "+grafana_token
}

for lambda_function in lambda_functions:

    source = sourceuid(str(lambda_function['account']), lambda_function['region'])
    folder = folderuid(str(lambda_function['account']))
    notify = alertuid(lambda_function['name'], lambda_function['region'], folder)

    if notify == None and folder != None:

        with open('lambda-error-alert.json') as f:
            j = json.load(f)

        j['title'] = lambda_function['name']
        j['folderUID'] = folder
        j['ruleGroup'] = lambda_function['region']
        j['data'][0]['datasourceUid'] = source
        j['data'][0]['model']['datasource']['uid'] = source
        j['data'][0]['model']['dimensions']['FunctionName'] = lambda_function['name']
        j['data'][0]['model']['sql']['where']['expressions'][0]['operator']['value'] = lambda_function['name']
        j['data'][0]['model']['sqlExpression'] = "SELECT SUM(Errors) FROM \"AWS/Lambda\" WHERE FunctionName = '"+lambda_function['name']+"'"

        url = grafana_url+'/api/v1/provisioning/alert-rules?query=json'
        response = requests.post(url, headers=headers, json=j)
        print(response.json())

        config = {}
        config['interval'] = 900
        url = grafana_url+'/api/v1/provisioning/folder/'+folder+'/rule-groups/'+lambda_function['region']+'?query=json'
        response = requests.put(url, headers=headers, json=config)

##### Delete Alerts

In [ ]:
import requests
import botoplus.organization as _org

selected = _org.selected()

ssm = selected.client('ssm')
grafana_url = ssm.get_parameter(Name='/grafana/url', WithDecryption=False)['Parameter']['Value']
grafana_token = ssm.get_parameter(Name='/grafana/token', WithDecryption=True)['Parameter']['Value']

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": "Bearer "+grafana_token
}
url = grafana_url+'/api/v1/provisioning/alert-rules?query=json'

response = requests.get(url, headers=headers)
j = response.json()

alert_rules = []

for alert in j:
    url = grafana_url+'/api/v1/provisioning/alert-rules/'+alert['uid']+'?query=json'
    response = requests.delete(url, headers=headers)
    print(response.text)